In [11]:
%load_ext autoreload
%autoreload 2
%load_ext snoop

import itertools
import pandas
import csv
import torch
import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
# from allennlp.data.token_indexers.bert_tokenizers import PretrainedBertIndexer

from awesome_glue.task import Task
from awesome_glue.utils import AttackMetric
from awesome_glue.config import Config

from luna import ram_write, chunks
from allennlpx import allenutil

from luna import ram_read

from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher

from collections import Counter, defaultdict
from tqdm import tqdm

from copy import deepcopy
from typing import List

import numpy
import torch
import numpy as np
from allennlp.common.util import JsonDict, sanitize
from allennlp.data.fields import TextField
from allennlp.data.token_indexers import ELMoTokenCharactersIndexer, TokenCharactersIndexer
from allennlp.data.tokenizers import Token
from allennlp.modules.text_field_embedders.text_field_embedder import TextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from luna import cast_list, lazy_property
from allennlp.data.vocabulary import Vocabulary, DEFAULT_OOV_TOKEN

from allennlpx.interpret.attackers.attacker import Attacker, DEFAULT_IGNORE_TOKENS
from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher
from allennlpx import allenutil
from itertools import product
from collections import defaultdict
import random

from functools import lru_cache
from allennlp.data.tokenizers import SpacyTokenizer
from luna import time_record

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The snoop extension is already loaded. To reload it, use:
  %reload_ext snoop


In [8]:
config = Config()
# config.finetunable=True
task = Task(config) # type: Task
task.from_pretrained()
config

cache for SST-spacy exists
*** load SST-spacy from cache cost 2.5 seconds
cache for 2f2a46e51336b27a7262689b00016955 exists
*** load 2f2a46e51336b27a7262689b00016955 from cache cost 0.0124 seconds
Restore model from checkpoint saved/models/SST-lstm-glove@0.ckpt.


Basic Args:
	--task_id=SST
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=SST-lstm-glove
	--mode=attack
	--adv_data=nogit/SST-bert.adv.tsv
	--aug_data=
	--attack_vectors=glove
	--attack_data_split=dev
	--attack_size=100
	--attack_gen_aug=False
	--attack_gen_adv=False
	--alchemist=False
	--seed=2
Deduced Args:
	--model_name=SST-lstm-glove
	--tokenizer=spacy

In [ ]:
task.vocab.get_index_to_token_vocabulary().values()

In [45]:
from allennlpx.interpret.attackers.pwws import PWWS
attacker = PWWS(task.predictor)
attacker.initialize()
sent = allenutil.as_sentence(task.dev_data[11])
print(sent)
result = attacker.attack_from_json({"sent": sent},
                                   field_to_change="sent",
#                                    ignore_tokens=forbidden_words,
#                                    forbidden_tokens=forbidden_words,
                                   max_change_num_or_ratio=0.25,
                                   measure='euc',
                                   topk=300,
#                                                    search_num=256
                                  )

... the film suffers from a lack of humor ( something needed to balance out the violence ) ...
2 film shoot 0.0009989738 0.0010770559
3 suffers support 0.029087007 0.033947527
5 a amp 1.3530254e-05 0.00017350912
6 lack miss 0.0007048249 0.0010660291
8 humor humor 0.0 -1.2874603e-05
11 needed ask 0.00010931492 0.00020360947
13 balance poise 0.0005479455 0.00019317865
14 out taboo 0.00027257204 9.047985e-05
16 violence fury 0.00023454428 0.00038200617


NameError: name 'sids' is not defined